In [26]:
import redis
redis_client = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)

print("Verbindung zu Redis hergestellt:", redis_client.ping())

Verbindung zu Redis hergestellt: True


# Aufgabe 1

In [27]:
redis_client.set("username", "alice")
print(redis_client.get("username"))
print(redis_client.exists("username"))

alice
1


# Aufgabe 2

In [28]:
redis_client.set("session_token", "xyz123", 10)
#oder
redis_client.set("session", "xyz123")
redis_client.expire("session_token", 10)

True

In [5]:
print(redis_client.ttl("session_token"))

8


# Aufgabe 3

In [6]:
redis_client.set("page_views", 1)

True

In [29]:
redis_client.incr("page_views", 1)
print(redis_client.get("page_views"))

5


# Aufgabe 4

In [30]:
redis_client.hset("user:1001", "name", "alice")
redis_client.hset("user:1001", "age", 30)
redis_client.hset("user:1001", "email", "alice@example.com")
print(redis_client.hgetall("user:1001"))
redis_client.hset("user:1001", "age", 40)
print(redis_client.hgetall("user:1001"))

{'logins': '2', 'name': 'alice', 'age': '30', 'email': 'alice@example.com'}
{'logins': '2', 'name': 'alice', 'age': '40', 'email': 'alice@example.com'}


# Aufgabe 5

In [46]:
redis_client.lpush("tasks", "send_email")
redis_client.lpush("tasks", "generate_report")
redis_client.lpush("tasks", "send_backup_db")
print(redis_client.rpop("tasks"))

send_backup_db


# Aufgabe 6

In [47]:
redis_client.sadd("unique_visiors", "192.168.0.1","192.168.0.2","192.168.0.1")
print(redis_client.scard("unique_visiors"))

2


# Aufgabe 7

In [48]:
redis_client.zadd("leaderboard", {"Alice":50, "Bob":70, "Carol":60})
redis_client.zincrby("leaderboard", 15, "Alice")
print(redis_client.zrevrange("leaderboard", 0, 100))

['Bob', 'Alice', 'Carol']


# Aufgabe 8

In [50]:
with redis_client.pipeline() as pipe:
    pipe.incr("coins", 10)
    pipe.time()
    results = pipe.execute()

last_update = results[1][0]
redis_client.set("last_update", last_update)
print(redis_client.get("last_update"))

1770625007


# Aufgabe 9

In [62]:
import random
import time
startWithPipe = time.time()
pipe = redis_client.pipeline()
for i in range(1, 1000):
    pipe.set("itemPipe:${i}", random.randint(1, 1000))
pipe.execute()
timeWithPipe = time.time() - startWithPipe

In [63]:
startWithoutPipe = time.time()
for i in range(1, 1000):
    redis_client.set("item:${i}", random.randint(1, 1000))
timeWithoutPipe = time.time() - startWithoutPipe
print("With Pipe ", timeWithPipe, " timeWithoutPipe ", timeWithoutPipe)

With Pipe  0.016187191009521484  timeWithoutPipe  0.26282548904418945


In [58]:
redis_client.delete("itemPipe:*")
redis_client.delete("item:*")

0

# Aufgabe 10

In [68]:
import time
def publisher():
    i = 0
    while True:
        redis_client.publish("channel0", i)
        i = i+1
        time.sleep(5)



In [69]:
def subscriber():
    subscriber = redis_client.pubsub()
    subscriber.subscribe("channel0")
    for message in subscriber.listen():
        if message['type'] == 'message':
            print(f"Received: {message['data']}")

In [71]:
import threading

threading.Thread(target=publisher, daemon=True).start()
threading.Thread(target=subscriber, daemon=True).start()

Received: 178
Received: 1
Received: 179
Received: 2
Received: 180
Received: 3
Received: 181
Received: 4
Received: 182
Received: 5
Received: 183
Received: 6
Received: 184
Received: 7
Received: 185
Received: 8
Received: 186
Received: 9
Received: 187
Received: 10
Received: 188
Received: 11
Received: 189
